<a href="https://polly.elucidata.io/manage/workspaces?action=open_polly_notebook&amp;source=github&amp;path=ElucidataInc%2Fpolly-python%2Fblob%2Fmain%2FDiscover%2Fsearch_across_all_omixatlas.ipynb&amp;kernel=elucidata%2FPython+3&amp;machine=medium" target="_parent"><img alt="Open in Polly" src="https://elucidatainc.github.io/PublicAssets/open_polly.svg"/></a>


# Prototype polly-python Notebook with examples showing how to query data across all OmixAtlas

For internal use only

Instructions:
1. Please run all code cells one by one.
2. User is required to enter an SQL query to search for the required datasets.

For any support or feedback, kindly reach out to either pawan.verma@elucidata.io or yogesh.lakhotia@elucidata.io


In [2]:
# please do not modify
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Install polly-python and joblib

In [ ]:
!sudo pip3 install polly-python joblib

In [ ]:
restartkernel() #Pause for a few seconds before the kernel is refreshed

In [1]:
# please do not modify
from IPython.display import HTML
HTML('''<script type="text/javascript"> Jupyter.notebook.kernel.execute("url = '" + window.location + "'", {}, {}); </script>''')

In [4]:
from polly.omixatlas import OmixAtlas
from polly.auth import Polly
from joblib import Parallel, delayed
import pandas as pd
import time
import re
import os

## Authenticate your Polly session using the token

In [5]:
AUTH_TOKEN=(os.environ['POLLY_REFRESH_TOKEN']) # Obtain authentication tokens
Polly.auth(AUTH_TOKEN)

In [19]:
omixatlas = OmixAtlas()

## Defining functions for searching across all OA

In [ ]:
def query_split(query):
    
    """
    Returns N SQL queries equal to the number of input indexes
    
    Extended description of function.
    
    Parameters:
    query (str): input SQL query to search across any given omixatlases
    
    Returns:
    q_list (list): list of SQL queries where each query queries on a single index only.
    
    """
    q_list = []
    
    query = query.replace("\n", "") #remove new line from query
    
    q_list = re.split('union', query, flags=re.IGNORECASE)
    q_list = [q.strip() for q in q_list]
    
    return q_list

def query_oa(query, version):
    
    """
    Returns a table containing metadata for all results based on the input SQL query
    
    Extended description of function.
    
    Parameters: 
    query (string): list of input SQL queries where each query is used to query across a single omixatlas
    version (string): The API version of data infra on which to query
    
    Returns:
    all_df (dataframe): DataFrame containing metadata of the resulting data.
    
    """

    all_df = omixatlas.query_metadata(query, query_api_version=version)
    return all_df
    

def parallel_query(query_list, version):
    
    """
    Performs a parallel execution by dividing the task among multiple threads rather than multiple CPUs
    
    Extended description of function.
    
    Parameters: 
    query_list (list): list of input SQL queries where each query is used to search across a single omixatlas
    version (string): The API version of data infra on which to query
    
    Returns:
    df (dataframe): DataFrame containing dataset level metadata of the resulting datasets.
    
    """

    final = []
    result = Parallel(n_jobs=4, prefer="threads")(delayed(query_oa)(query, version) for query in query_list)

    for that_dict in result:
        if isinstance(that_dict, pd.DataFrame):
            final.append(that_dict)
    
    df = pd.concat(final)
    return df

def empty_df():

    """
    Creates an empty dataframe with a message when no data is returned
    
    Extended description of function.
    
    Returns:
    empty_df (dataframe): An empty dataframe with a message.
    
    """

    empty_df = pd.DataFrame()
    data = pd.DataFrame({"Message": "No Data to show!"}, index=[0])
    empty_df = empty_df.append(data)
    return empty_df

def query_all_oa(sql, api_ver):

    """
    Entry point for the cross omixatlas querying app.
    
    Extended description of function.
    
    Parameters: 
    sql (string): Input SQL query
    api_ver (string): The API version of data infra on which to query
    
    Returns:
    result_df (dataframe): DataFrame containing dataset level metadata of the resulting datasets.
    status (string): String containing messages for the user
    time_elapsed (string): Execution wall-time in seconds
    """
    
    start_time = time.time()
    result_df = pd.DataFrame()
    
    if('union' in sql.lower()):
        try:
            query_list = query_split(sql)
            result_df = parallel_query(query_list, api_ver)
            if result_df.empty:
                status = 'Query successful, No datasets were returned'
                result_df = empty_df()
            else:
                status = 'Query Successful'
                
        except Exception as e:
            status = repr(e)
            result_df = empty_df()
    else:
        try:
            result_df = query_oa(sql, api_ver)
            if result_df.empty:
                status = 'Query successful, No datasets were returned'
                result_df = empty_df()
            else:
                status = 'Query Successful'
                
        except Exception as e:
            status = repr(e)
            result_df = empty_df()
    
    time_elapsed = str("Elapsed time = --- %s seconds ---" % (time.time() - start_time))
    return(result_df, status, time_elapsed)

## Example query

In [ ]:
query = f"""SELECT dataset_id, description, disease, tissue FROM geo.datasets WHERE data_type = 'Transcriptomics'
UNION SELECT dataset_id, description, disease, tissue FROM tcga.datasets WHERE data_type = 'Mutation'"""

version = 'v2'

result, status, wall_time = query_all_oa(query, version)

print(wall_time)
print(status)
result